In [1]:
import os
import re
import pandas as pd
from llama_cpp import Llama  # 🚀 Modèle LLM local

# 📌 Charger le modèle LLM (Mistral 7B GGUF)
llm = Llama(model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_ctx=2048)

OSError: exception: access violation reading 0x0000000000000000

In [42]:
import os
import re
import pandas as pd

# 📌 Fonction pour extraire l'année d'un nom de fichier
def extract_year_from_filename(filename):
    match = re.search(r"(20\d{2})", filename)
    return int(match.group(1)) if match else None

# 📌 Dossier contenant les fichiers CSV
data_folder = "data"

def search_f1_data(question):
    question = question.lower()
    files_to_return = []  # Liste des fichiers CSV à retourner

    # 📌 Vérifier s'il y a une année spécifique dans la question
    match = re.search(r"(20\d{2})", question)
    year = int(match.group(1)) if match else None

    # 📌 Vérifier si l'utilisateur demande les 5 dernières années
    if any(keyword in question for keyword in ["5 dernières années", "dernières années", "cinq ans","l'année prochaine"]):
        if year:
            start_year = max(2020, year - 4)  # Assurer qu'on ne descend pas sous 2020
        else:
            start_year = 2020  # Par défaut, on prend les 5 dernières années disponibles
        years = list(range(start_year, start_year + 5))
        print("5 dernières années:", years)
    else:
        years = [year] if year else []
        print("année:", years)

    # 📌 Identifier les fichiers à récupérer
    if "classement" or "champion" or "pilote" in question:
        print("classement")
        files_to_return = [f"classement/classement_f1_{y}.csv" for y in years]
        print(files_to_return)
    elif "calendrier" or "date" in question:
        files_to_return = [f"calendrier/calendrier_f1_{y}.csv" for y in years]
    elif "circuit" in question:
        files_to_return = [f"circuits/circuits_f1_{y}.csv" for y in years]
    elif "constructeur" in question:
        files_to_return = [f"constructeurs/constructeurs_f1_{y}.csv" for y in years]
# 📌 Charger et concaténer les fichiers trouvés
    data_results = []
    for file_key in files_to_return:
        file_path = os.path.join(data_folder, file_key)
        print("Fichier:", file_path)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            data_results.append(f"📂 **Données de {file_key}**\n{df.to_string(index=False)}\n\n")

    # 📌 Retourner les résultats concaténés
    if data_results:
        
        return "\n".join(data_results)

    return "Désolé, je n'ai pas trouvé de données correspondantes."

In [43]:
def generate_response(question, data_results):
    # 🔥 Limite la taille des données envoyées
    if len(data_results) > 1500:
        data_results = data_results[:1500]  # Garde uniquement les premiers 1500 caractères
    
    prompt = f"""
    [INST] 
    Tu es un expert en Formule 1. Un utilisateur te pose la question :
    "{question}"

    Voici les données brutes extraites des fichiers CSV :
    {data_results}

    Analyse les données et donne une réponse claire et concise.
    [/INST]
    """

    response = llm(prompt, max_tokens=256, stream=True)
    for chunk in response:
        print(chunk["choices"][0]["text"], end="", flush=True)
    print()


In [44]:

while True:
        user_input = input("Pose une question sur la F1 : ")
        if user_input.lower() == "exit":
            break

        data_results = search_f1_data(user_input)
        generate_response(user_input, data_results)
